### Agent-Lab: Multi-Agent -> Coder

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture
import json
import os
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

os.chdir("..")
load_dotenv()
nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### XAI Grok Coder

In [2]:
# Create Workflow
xai_agent = experiment_utils.create_xai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="grok-code-fast", api_key=os.getenv("XAI_API_KEY")
)
xai_coder_agent = container.coordinator_planner_supervisor_agent()
xai_workflow_builder = xai_coder_agent.get_workflow_builder(xai_agent["id"])
xai_workflow = xai_workflow_builder.compile(checkpointer=checkpointer)

In [3]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading. Make sure to include code examples in the final report.",
    agent_id=xai_agent["id"],
)

inputs = xai_coder_agent.get_input_params(message, schema="public")
config = xai_coder_agent.get_config(xai_agent["id"])
result = xai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = xai_coder_agent.format_response(result)

In [4]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "Generate a simple Hello World program using FastAPI in Python, formatted as a markdown text report with code examples included, without handling library setup.",
  "title": "Creating a FastAPI Hello World Application",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Generate FastAPI Hello World Code",
      "description": "Use coder to create a simple FastAPI hello world Python script and output it in a markdown-formatted report including code examples."
    },
    {
      "agent_name": "reporter",
      "title": "Compile Final Report",
      "description": "Use reporter to write a professional final report incorporating the markdown-formatted code from the coder's output."
    }
  ]
}
```

In [5]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

# FastAPI Hello World Example

This report provides a simple "Hello World" implementation using FastAPI in Python. FastAPI is a modern, fast web framework for building APIs with Python 3.7+ based on standard Python type hints.

## Requirements

- Python 3.7 or later
- FastAPI installed (`pip install fastapi`)
- Uvicorn for running the server (`pip install uvicorn`)

## Code Example

Below is the complete Python code for a basic FastAPI application that returns a "Hello World" JSON response when accessed via the root endpoint (`/`).

```python
from fastapi import FastAPI

# Create a FastAPI instance
app = FastAPI()

# Define a GET endpoint at the root path "/"
@app.get("/")
async def read_root():
    return {"Hello": "World"}
```

## How to Run

1. Save the code above to a file, e.g., `main.py`.
2. Run the application using Uvicorn:
   ```
   uvicorn main:app --reload
   ```
3. Open your browser and navigate to `http://127.0.0.1:8000/` to see the JSON response: `{"Hello": "World"}`.

## Additional Notes

- This is a minimal example. FastAPI supports automatic API documentation at `http://127.0.0.1:8000/docs` (Swagger UI) and `http://127.0.0.1:8000/redoc` (ReDoc).
- For production, consider adding more endpoints, request/response models, and error handling as needed.

---
### Anthropic Coder

In [6]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-3-5-haiku-latest", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_coder_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_coder_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [7]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading. Make sure to include code examples in the final report.",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_coder_agent.get_input_params(message, schema="public")
config = anthropic_coder_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_coder_agent.format_response(result)

In [8]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "I'll use the coder agent to generate a simple Hello World FastAPI application and provide a comprehensive code example with markdown-formatted documentation.",
  "title": "Generate FastAPI Hello World Example",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Create FastAPI Hello World Code",
      "description": "Write a simple FastAPI application that returns a hello world message. Generate a markdown-formatted code report with explanations."
    },
    {
      "agent_name": "reporter",
      "title": "Compile Final Report",
      "description": "Create a professional report presenting the FastAPI hello world code with detailed explanations and best practices."
    }
  ]
}
```

In [9]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**



# FastAPI Hello World Application

## Overview
This is a simple FastAPI application that demonstrates a basic endpoint returning a "Hello, World!" message. FastAPI is a modern, fast web framework for building APIs with Python 3.6+ based on standard Python type hints.

## Code Example

```python
from fastapi import FastAPI

# Create a FastAPI application instance
app = FastAPI()

# Define a simple GET route that returns a hello world message
@app.get("/")
async def read_root():
    """
    Root endpoint that returns a hello world message.
    
    Returns:
        dict: A dictionary containing a greeting message
    """
    return {"message": "Hello, World!"}

# Optional: Additional endpoint example
@app.get("/hello/{name}")
async def hello_name(name: str):
    """
    Personalized greeting endpoint that takes a name parameter.
    
    Args:
        name (str): The name to include in the greeting
    
    Returns:
        dict: A personalized greeting message
    """
    return {"message": f"Hello, {name}!"}
```

## Key Components Explained

1. **Imports**
   - `from fastapi import FastAPI`: Imports the main FastAPI class

2. **Application Instance**
   - `app = FastAPI()`: Creates the main application object

3. **Route Decorators**
   - `@app.get("/")`: Defines a GET route for the root path
   - `@app.get("/hello/{name}")`: Creates a dynamic route with a path parameter

4. **Async Functions**
   - Using `async def` allows for asynchronous request handling
   - Returns simple dictionaries that will be automatically converted to JSON responses

## Running the Application
To run this application, you would typically use Uvicorn:
```bash
uvicorn main:app --reload
```

## Best Practices
- Use type hints for parameters
- Provide docstrings for routes
- Keep routes simple and focused
- Use async functions for better performance

## Potential Enhancements
- Add error handling
- Implement more complex routing
- Add request validation
- Integrate with databases or other services

## Notes
This is a minimal example to demonstrate the basic structure of a FastAPI application. In a real-world scenario, you would add more robust error handling, middleware, and additional endpoints.

---
### OpenAI Coder

In [10]:
# Create Workflow
openai_agent = experiment_utils.create_openai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="gpt-5-nano", api_key=os.getenv("OPENAI_API_KEY")
)
openai_coder_agent = container.coordinator_planner_supervisor_agent()
openai_workflow_builder = openai_coder_agent.get_workflow_builder(openai_agent["id"])
openai_workflow = openai_workflow_builder.compile(checkpointer=checkpointer)

In [11]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading.",
    agent_id=openai_agent["id"],
)

inputs = openai_coder_agent.get_input_params(message, schema="public")
config = openai_coder_agent.get_config(openai_agent["id"])
result = openai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = openai_coder_agent.format_response(result)

In [12]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "User wants a simple Hello World application using FastAPI in Python. No external setup beyond generating the code; the output should be a Markdown-formatted text report containing the code snippet.",
  "title": "Plan to generate a simple FastAPI Hello World app in Python with Markdown-formatted code",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Create minimal FastAPI Hello World script and Markdown-formatted snippet",
      "description": "Produce a Python file (e.g., main.py) implementing a FastAPI app with a root endpoint that returns a Hello World response. Output the code inside a Markdown code block, and include a brief usage section with the uvicorn command (e.g., uvicorn main:app --reload --port 8000)."
    },
    {
      "agent_name": "reporter",
      "title": "Publish final professional Markdown report",
      "description": "Take the coder's Markdown content and assemble a clean, reader-friendly Markdown document containing an introduction, the FastAPI code block, and run instructions. Deliver as the final answer."
    }
  ]
}
```

In [13]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

# Executive Summary
This report presents a minimal FastAPI-based Hello World application in Python. It provides the complete code (main.py), a single root endpoint that returns a Hello, World! message, and simple run instructions via uvicorn or direct execution. The solution is intentionally compact and easy to extend for further development.

# Key Findings
- Endpoint: GET / returns a JSON message: {"message": "Hello, World!"}
- Source file: main.py containing the FastAPI app and route.
- Run options:
  - Quick development run with uvicorn: uvicorn main:app --reload --port 8000
  - Direct execution: python main.py (utilizes the __main__ block to start uvicorn)
- Dependencies mentioned: fastapi and uvicorn (installation example provided as pip install fastapi uvicorn)
- The approach is designed to be minimal and easily extensible.

# Detailed Analysis
Code structure (as provided):
- Imports FastAPI from fastapi
- Creates an app instance: app = FastAPI()
- Defines a route:
  - @app.get("/")
  - async def read_root(): return {"message": "Hello, World!"}
- Optional execution path:
  - if __name__ == "__main__":
      import uvicorn
      uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

Key behavior:
- The root path ("/") responds with a JSON object containing the message "Hello, World!".
- The app is asynchronous, leveraging FastAPI's async function for the route.
- The code includes guidance to run via uvicorn with reload and a port of 8000, and an alternative to run directly through Python, which triggers uvicorn.

Usage instructions (as provided):
- Install dependencies (not shown in the code snippet): pip install fastapi uvicorn
- Run with uvicorn (recommended for development):
  - uvicorn main:app --reload --port 8000
- Or run directly (uses the __main__ block):
  - python main.py

Endpoint details:
- GET http://localhost:8000/
- Response body: { "message": "Hello, World!" }

Notes:
- The code is intentionally minimal and easy to extend for additional endpoints or functionality.

# Conclusions and Recommendations
- Conclusion: The provided code delivers a concise and functional Hello World API using FastAPI. It is suitable for quick demonstrations, learning, or as a starting point for a broader API.
- Recommendations:
  - Use the uvicorn-based run command for development and testing to leverage auto-reload.
  - Extend the app by adding additional endpoints or business logic as needed.
  - Keep the explicit __main__ block to support running the app directly via Python.

# Data Sources
- Code snippet: main.py implementing the FastAPI app and root endpoint.
- Run instructions and endpoint example (as described in the provided text). Information not provided elsewhere (e.g., production deployment steps) is explicitly labeled as not provided.